In [ ]:
%%capture
!pip -q install openai
!pip -q install langchain-core
!pip -q install langchain-community
!pip -q install sentence-transformers
!pip -q install langchain-huggingface
!pip -q install langchain-experimental
!pip -q install langchainhub
!pip -q install langchain-openai

# for wikipedia as a tool
!pip -q install -U wikipedia

# for google-search as a tool
!pip -q install google-search-results

In [ ]:
import os
from google.colab import userdata

import openai
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import load_tools, create_react_agent, AgentExecutor
from langchain.tools.render import render_text_description
from langchain_community.utilities import SerpAPIWrapper

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# We are going to use this language model as the reasoning engine of the agent to connect to other data sources and perform computation
# so we want this reasoning engine to be as presice as possible.
# Hence, temperature is set to zero to get rid of any randomness.

In [ ]:
import datetime as dt

def timefunc(text: str) -> str:
    dt_india = dt.datetime.utcnow() + dt.timedelta(hours=5, minutes=30)
    return dt_india.strftime('%Y-%m-%d %H:%M:%S.%f')

In [ ]:
# Create the tool to pass to an agent

from langchain_core.tools import Tool

timetool = Tool(
    name="time_tool",
    description="""Returns current date and time, use this for any \
    questions related to knowing current date and Indian time. \
    The input should always be an empty string, \
    and this function will always return current  \
    date and time.""",
    func = timefunc,
)

In [ ]:
tools = load_tools([], llm=llm) + [timetool, ]
tools

In [ ]:
# Create the ReAct agent using the create_react_agent function

agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
    stop_sequence=True,
)

In [ ]:
# Create an agent executor from the agent and tools

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [ ]:
result = agent_executor.invoke(
    {
        "input": "What is the current time?"
    }
)
result